# Библиотеки и CUDA

In [6]:
# импортируем библиотеку torch
import torch

In [7]:
# обработка ошибок и предупреждений
import warnings
warnings.filterwarnings('ignore')

In [8]:
# вывод всех графиков и прочих картинок в самом файле .ipynb
%matplotlib inline

In [9]:
# для работы с данными в формате DataFrame (анализ; подготовка набора данных)
import pandas as pd 

# Модуль А. Анализ и предобработка данных

## Анализ данных

### Датасет LJSpeech-1.1

*Исходные данные представлены в папке "LJSpeech-1.1". В ней содержатся следующие файлы:*

* Файл "metadata.csv", в котором хранятся тексты 

* Файл README с описанием набора данных

* Папка "wavs", в которой хранятся аудиофайлы в формате .wav с голосовой записью текстов из "metadata.csv".  *Каждому аудиофайлу соответствует текст с таким же значением в первой колонке в файле .csv*

____

Для анализа файла "metadata.csv" я сделаю из него датафрейм с помощью библиотеки pandas и буду анализировать так же с помощью методов из этой библиотеки

*ps: датафрейм где-то будет сокращён до дф*

In [13]:
# создаём дф
df_ishod = pd.read_csv(r'LJSpeech-1.1\metadata.csv', sep = '|', header=None)
df_ishod # вывод дф

,0,1,2
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...
...,...,...,...
13095,LJ050-0274,made certain recommendations which it believes...,made certain recommendations which it believes...
13096,LJ050-0275,materially improve upon the procedures in effe...,materially improve upon the procedures in effe...
13097,LJ050-0276,"As has been pointed out, the Commission has no...","As has been pointed out, the Commission has no..."
13098,LJ050-0277,with the active cooperation of the responsible...,with the active cooperation of the responsible...


В первой колонке датафрейма хранятся названия файлов из папки "wavs", во второй и третьей колонках - тексты из аудиофайлов.

Называем колонки в датафрейме в соответствии с тем, какая информация в них есть:

In [14]:
df_ishod.columns = ['file_name', 'text1', 'text2'] # добавляем колонки
df_ishod # выводим дф

,file_name,text1,text2
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...
...,...,...,...
13095,LJ050-0274,made certain recommendations which it believes...,made certain recommendations which it believes...
13096,LJ050-0275,materially improve upon the procedures in effe...,materially improve upon the procedures in effe...
13097,LJ050-0276,"As has been pointed out, the Commission has no...","As has been pointed out, the Commission has no..."
13098,LJ050-0277,with the active cooperation of the responsible...,with the active cooperation of the responsible...


В датафрейме 13100 строчек, столько же, сколько и аудиофайлов.

* В первую очередь посмотрим, отличаются ли тексты в колонках text1 и text2

In [ ]:
list_dub = [] # список для индексов строк с разными текстами в колонках

# проходимся по всем строчкам в дф
for i in range(len(df_ishod)):
    # проверяем равны ли данные
    if df_ishod['text1'].iloc[i] != df_ishod['text2'].iloc[i]:
        print(i) # если нет, то выводим индекс строки
        list_dub.append(i) # и добавляем его в список list_dub

print(f'Количество строк с неодинаковыми текстами: {len(list_dub)}\nИндексы строк с неодинаковыми текстами: {list_dub}')

6
23
30
33
37
44
48
76
90
91
92
121
186
193
194
195
196
198
199
200
201
205
206
207
209
210
211
214
215
217
218
219
220
221
233
237
240
247
263
274
302
303
304
306
308
309
310
311
313
314
321
323
331
332
335
338
339
340
341
342
343
344
348
360
361
363
378
383
399
409
424
425
427
430
432
448
451
452
454
456
476
482
488
498
508
523
557
558
565
572
573
574
576
577
593
596
665
699
715
717
745
798
809
810
816
871
881
894
896
908
945
948
954
959
961
972
973
976
980
981
989
1014
1021
1022
1056
1084
1097
1111
1118
1119
1120
1130
1147
1149
1176
1192
1193
1194
1196
1203
1252
1253
1256
1259
1260
1264
1285
1305
1319
1334
1338
1343
1359
1361
1368
1369
1371
1374
1400
1405
1419
1425
1426
1500
1537
1538
1539
1655
1726
1803
1824
1825
1853
1856
1868
1907
1929
1933
1935
1939
1949
1968
1969
1978
1981
1984
1987
1988
1990
1998
2004
2018
2042
2061
2112
2117
2119
2124
2149
2154
2158
2163
2165
2233
2287
2317
2446
2453
2455
2456
2458
2459
2460
2461
2462
2464
2465
2474
2475
2483
2488
2503
2535
2553
2570
2586
259

In [ ]:
df_dub = pd.DataFrame(columns=['file_name', 'text1', 'text2']) # дф для строк с разными текстами
# проходимся по всем строкам
for i in range(len(df_ishod)):
    # если индекс есть в листе, то добавляем соответствующую строку в df_dub
    if i in list_dub:
        df_dub = pd.concat([df_dub, df_ishod.loc[[i]]], ignore_index=True)

df_dub # выводим дф

,file_name,text1,text2
0,LJ001-0007,"the earliest book printed with movable types, ...","the earliest book printed with movable types, ..."
1,LJ001-0024,But the first Bible actually dated (which also...,But the first Bible actually dated (which also...
2,LJ001-0031,In 1465 Sweynheim and Pannartz began printing ...,In fourteen sixty-five Sweynheim and Pannartz ...
3,LJ001-0034,"They printed very few books in this type, thre...","They printed very few books in this type, thre..."
4,LJ001-0038,while in 1470 at Paris Udalric Gering and his ...,while in fourteen seventy at Paris Udalric Ger...
...,...,...,...
1516,LJ050-0236,"Thus, in the 4 months following the assassinat...","Thus, in the four months following the assassi..."
1517,LJ050-0237,"the FBI, on 16 separate occasions, supplied a ...","the FBI, on sixteen separate occasions, suppli..."
1518,LJ050-0239,"From February 11 through June 30, 1964,","From February eleven through June thirty, nine..."
1519,LJ050-0240,"the Service had the advantage of 9,500 hours o...",the Service had the advantage of nine thousand...


* Отличия в текстах по колонкам text1 и text2 заключаются в том, что в колонке text1 числовые значения записаны цифрами, а в колонке text2 они записаны словами.